# Original Code

In [ ]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import AES
import pyAesCrypt
import string
import secrets




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   

port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

start1 = time.time()
#start = time.time()    
#Registration
#Di=int(input("Di input:")) # Di input
Di = 12345
#PWi=input("PWi input:") # PWi input
PWi = "12345"

Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi

end1 = time.time()

#start = time.time()
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host_name, port))
    s.listen(5)
    while True:
        conn,addr=s.accept()
        with conn:
            #print("Connected to",addr)
            while True:
                data=conn.recv(1024)
                if not data:
                    break
                # print("Received from",data.decode())
                sent=temp.encode()
                try:
                    conn.sendall(sent)
                except ConnectionResetError as e:
                    break
        break

#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        t_=data.decode()
        break
        
        
start2 = time.time()
        
l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
print(k)
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

#File Encryption
#alphabet = string.ascii_letters + string.digits
#password = ''.join(secrets.choice(alphabet) for i in range(256))
#pyAesCrypt.encryptFile("Store_file.txt", "Encrypted.txt.aes", password)


#end = time.time()
#totaltime = end-start
#print("Registration time: ", totaltime)

end2 = time.time()
totaltime = (end1-start1) + (end2-start2)
print("Registration time: ", totaltime)


start3 = time.time()
#start=time.time()
#Authentication
#Di_auth=int(input("Di input Authentication:"))
Di_auth = 1234
#PWi_auth=input("Enter the PWi@Di_auth:")
PWi_auth = "1234@1234"
r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0
#File Decryption
#pyAesCrypt.decryptFile("Encrypted.txt.aes", "Decrypted.txt", password)

file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         
#if flag == 0:
#    print('String', s , 'Not Found')
#else:
#    print('String', s, 'Found In Line', index)
file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    #print(da,pa,ka,xa,T_sa,Via)
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    #print("Vi_prime:",Vi_prime) # Calculate the value of Vi_prime
    check=int(ka)^int(T_sa)
    #print("check:",check) # Used to check whether the calculated Vi_prime is same is ka^T_sa

    b = str(Vi_prime) + str(N_auth)
    #print("b value: ", b)
    # Encryption starts
    block_size = 16
    pad = lambda s: s + (block_size - len(s) % block_size) * chr(block_size - len(s) % block_size)
    iv = Random.new().read(AES.block_size) # Random IV

    def get_private_key(secret_key, salt):
        return hashlib.pbkdf2_hmac('SHA256', secret_key.encode(), salt.encode(), 65536, 32)

    def encrypt_with_AES(message, secret_key, salt):
        private_key = get_private_key(secret_key, salt)
        message = pad(message)
        cipher = AES.new(private_key, AES.MODE_CBC, iv)
        cipher_bytes = base64.b64encode(iv + cipher.encrypt(message))
        return bytes.decode(cipher_bytes)
    
    secret_key = str(Krs)
    salt = "salt"
    plain_text = str(b)
    
    cipher = encrypt_with_AES(plain_text, secret_key, salt)
    #print("Cipher: " + cipher)
    
    session_device = str(N_auth) + str(Krs)
    print("session device",session_device)
    
    end3 = time.time()
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    #port = int(input("Enter the port number:"))
    port = 4040
    host_name = socket.gethostname()
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host_name, port))
        s.listen(5)
        while True:
            conn,addr=s.accept()
            with conn:
                #print("Connected to",addr)
                while True:
                    data=conn.recv(1024)
                    if not data:
                        break
                    #print("Received from",data.decode())
                    sent=temp.encode()
                    try:
                        conn.sendall(sent)
                    except ConnectionResetError as e:
                        break
            break

#receving h_n from gateway
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

start4 = time.time()
l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")

end4 = time.time()
#end = time.time()
totaltime = (end3-start3) + (end4-start4)
#totaltime = end-start
print("Authentication time: ", totaltime)

# DES

In [ ]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import DES3
import pyAesCrypt
import string
import secrets




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   


    
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break



start1 = time.time()
#start = time.time()
#Registration
#Di=int(input("Di input:")) # Di input
Di = 12345
#PWi=input("PWi input:") # PWi input
PWi = "12345"
Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi

end1 = time.time()

#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host_name, port))
    s.listen(5)
    while True:
        conn,addr=s.accept()
        with conn:
            #print("Connected to",addr)
            while True:
                data=conn.recv(1024)
                if not data:
                    break
                # print("Received from",data.decode())
                sent=temp.encode()
                try:
                    conn.sendall(sent)
                except ConnectionResetError as e:
                    break
        break

#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        t_=data.decode()
        break


start2 = time.time()
       
l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

#File Encryption
#alphabet = string.ascii_letters + string.digits
#password = ''.join(secrets.choice(alphabet) for i in range(256))
#pyAesCrypt.encryptFile("Store_file.txt", "Encrypted.txt.aes", password)


end2 = time.time()
#end = time.time()
totaltime = (end1-start1) + (end2-start2)
#totaltime = end-start
print("Registration time: ", totaltime)


start3 = time.time()
#start = time.time()
#Authentication
#Di_auth=int(input("Di input Authentication:"))
Di_auth = 1
#PWi_auth=input("Enter the PWi@Di_auth:")
PWi_auth = "tarun@1"
r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0
#File Decryption
#pyAesCrypt.decryptFile("Encrypted.txt.aes", "Decrypted.txt", password)

file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         
#if flag == 0:
#    print('String', s , 'Not Found')
#else:
#    print('String', s, 'Found In Line', index)
file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    #print(da,pa,ka,xa,T_sa,Via)
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    #print("Vi_prime:",Vi_prime) # Calculate the value of Vi_prime
    check=int(ka)^int(T_sa)
    #print("check:",check) # Used to check whether the calculated Vi_prime is same is ka^T_sa

    b = str(Vi_prime) + str(N_auth)
    #print("b value: ", b)
    
    
    # Encryption starts
    BS = DES3.block_size
    pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)



    def encrypt(text, key):
        text = pad(text)
        cipher = DES3.new(key,DES3.MODE_ECB)
        m = cipher.encrypt(text)
        m = base64.b64encode(m)
        return m.decode('utf-8')
 

    secret_key = str(Krs)
    plain_text = str(b)
    
    if(len(secret_key)!=24):
        secret_key = secret_key.zfill(24)
    
    cipher = encrypt(plain_text, secret_key)
    #print("Cipher: " + cipher)
    
    session_device = str(N_auth) + str(Krs)
    #print("session device",session_device)
    
    end3 = time.time()
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    #port = int(input("Enter the port number:"))
    port = 4040
    host_name = socket.gethostname()
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host_name, port))
        s.listen(5)
        while True:
            conn,addr=s.accept()
            with conn:
                #print("Connected to",addr)
                while True:
                    data=conn.recv(1024)
                    if not data:
                        break
                    #print("Received from",data.decode())
                    sent=temp.encode()
                    try:
                        conn.sendall(sent)
                    except ConnectionResetError as e:
                        break
            break

#receving h_n from gateway
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

start4 = time.time()
l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")
end4 = time.time()
#end = time.time()
#totaltime = end-start
totaltime = totaltime = (end3-start3) + (end4-start4)
print("Authentication time: ", totaltime)

# RC4

In [ ]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import ARC4 as rc4cipher
import pyAesCrypt
import string
import secrets




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   

    
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break
        
        
start1 = time.time()
#start = time.time()
#Registration
#Di=int(input("Di input:")) # Di input
Di = 12345
#PWi=input("PWi input:") # PWi input
PWi = "12345"
Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi


end1 = time.time()


#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host_name, port))
    s.listen(5)
    while True:
        conn,addr=s.accept()
        with conn:
            #print("Connected to",addr)
            while True:
                data=conn.recv(1024)
                if not data:
                    break
                # print("Received from",data.decode())
                sent=temp.encode()
                try:
                    conn.sendall(sent)
                except ConnectionResetError as e:
                    break
        break

#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        t_=data.decode()
        break


start2 = time.time()
l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

#File Encryption
#alphabet = string.ascii_letters + string.digits
#password = ''.join(secrets.choice(alphabet) for i in range(256))
#pyAesCrypt.encryptFile("Store_file.txt", "Encrypted.txt.aes", password)


#end = time.time()
end2 = time.time()
totaltime = (end1-start1) + (end2-start2)
#totaltime = end-start
print("Registration time: ", totaltime)


start3 = time.time()
start = time.time()
#Authentication
#Di_auth=int(input("Di input Authentication:"))
Di_auth = 1234
#PWi_auth=input("Enter the PWi@Di_auth:")
PWi_auth = "1234@1234"
r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0
#File Decryption
#pyAesCrypt.decryptFile("Encrypted.txt.aes", "Decrypted.txt", password)

file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         
#if flag == 0:
#    print('String', s , 'Not Found')
#else:
#    print('String', s, 'Found In Line', index)
file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    #print(da,pa,ka,xa,T_sa,Via)
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    #print("Vi_prime:",Vi_prime) # Calculate the value of Vi_prime
    check=int(ka)^int(T_sa)
    #print("check:",check) # Used to check whether the calculated Vi_prime is same is ka^T_sa

    b = str(Vi_prime) + str(N_auth)
    #print("b value: ", b)
    # Encryption starts
    def rc4_algorithm(encrypt_or_decrypt, data, key1):
        if encrypt_or_decrypt == "encrypt":
            key = bytes(key1, encoding='utf-8')
            enc = rc4cipher.new(key)
            res = enc.encrypt(data.encode('utf-8'))
            res=base64.b64encode(res)
            res = str(res,'utf8')
            return res
    
    secret_key = str(Krs)
    plain_text = str(b)
    
    data = plain_text
    key = secret_key
    res ='dLzb09qr'
    
    cipher = rc4_algorithm('encrypt',data,key)
    #print("Cipher: " + cipher)
    
    session_device = str(N_auth) + str(Krs)
    #print("session device",session_device)
    
    end3 = time.time()
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    #port = int(input("Enter the port number:"))
    port = 4040
    host_name = socket.gethostname()
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host_name, port))
        s.listen(5)
        while True:
            conn,addr=s.accept()
            with conn:
                #print("Connected to",addr)
                while True:
                    data=conn.recv(1024)
                    if not data:
                        break
                    #print("Received from",data.decode())
                    sent=temp.encode()
                    try:
                        conn.sendall(sent)
                    except ConnectionResetError as e:
                        break
            break

#receving h_n from gateway
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

        
start4 = time.time()        
l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")

#end = time.time()
end4 = time.time()
totaltime = (end3-start3) + (end4-start4)
#totaltime = end-start
print("Authentication time: ", totaltime)

# with new server

In [ ]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import AES
import pyAesCrypt
import string
import secrets




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   

host_name = "DESKTOP-8RJUAE7"
s = socket.socket()        
port = 4040            
s.connect((host_name, port))
temp = s.recv(1024).decode()
s.close()  

#start1 = time.time()
start = time.time()    
#Registration
#Di=int(input("Di input:")) # Di input
Di = 12345
#PWi=input("PWi input:") # PWi input
PWi = "12345"

Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi

#end1 = time.time()

#start = time.time()
#port = int(input("Enter the port number:"))

host_name = "DESKTOP-8RJUAE7"
s = socket.socket()
port = 4040
s.bind((host_name, port))
s.listen(5)
while True:
    c, addr = s.accept()
    c.send(temp.encode())
    c.close()
    break

#port = int(input("Enter the port number:"))
host_name = "DESKTOP-8RJUAE7"
print(type(host_name))
s = socket.socket()        
port = 4040            
s.connect((host_name, port))
t_ = s.recv(1024).decode()
s.close()  
        
        
#start2 = time.time()
        
l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

#File Encryption
#alphabet = string.ascii_letters + string.digits
#password = ''.join(secrets.choice(alphabet) for i in range(256))
#pyAesCrypt.encryptFile("Store_file.txt", "Encrypted.txt.aes", password)


end = time.time()
totaltime = end-start
print("Registration time: ", totaltime)

#end2 = time.time()
#totaltime = (end1-start1) + (end2-start2)
#print("Registration time: ", totaltime)


#start3 = time.time()
start=time.time()
#Authentication
#Di_auth=int(input("Di input Authentication:"))
Di_auth = 1234
#PWi_auth=input("Enter the PWi@Di_auth:")
PWi_auth = "1234@1234"
r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0
#File Decryption
#pyAesCrypt.decryptFile("Encrypted.txt.aes", "Decrypted.txt", password)

file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         
#if flag == 0:
#    print('String', s , 'Not Found')
#else:
#    print('String', s, 'Found In Line', index)
file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    #print(da,pa,ka,xa,T_sa,Via)
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    #print("Vi_prime:",Vi_prime) # Calculate the value of Vi_prime
    check=int(ka)^int(T_sa)
    #print("check:",check) # Used to check whether the calculated Vi_prime is same is ka^T_sa

    b = str(Vi_prime) + str(N_auth)
    #print("b value: ", b)
    # Encryption starts
    block_size = 16
    pad = lambda s: s + (block_size - len(s) % block_size) * chr(block_size - len(s) % block_size)
    iv = Random.new().read(AES.block_size) # Random IV

    def get_private_key(secret_key, salt):
        return hashlib.pbkdf2_hmac('SHA256', secret_key.encode(), salt.encode(), 65536, 32)

    def encrypt_with_AES(message, secret_key, salt):
        private_key = get_private_key(secret_key, salt)
        message = pad(message)
        cipher = AES.new(private_key, AES.MODE_CBC, iv)
        cipher_bytes = base64.b64encode(iv + cipher.encrypt(message))
        return bytes.decode(cipher_bytes)
    
    secret_key = str(Krs)
    salt = "salt"
    plain_text = str(b)
    
    cipher = encrypt_with_AES(plain_text, secret_key, salt)
    #print("Cipher: " + cipher)
    
    session_device = str(N_auth) + str(Krs)
    #print("session device",session_device)
    
    #end3 = time.time()
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    #port = int(input("Enter the port number:"))
    host_name = "DESKTOP-8RJUAE7"
    s = socket.socket()
    port = 4040
    s.bind((host_name, port))
    s.listen(5)
    while True:
        c, addr = s.accept()
        c.send(temp.encode())
        c.close()
        break

#receving h_n from gateway
#port = int(input("Enter the port number:"))
host_name = "DESKTOP-8RJUAE7"
s = socket.socket()        
port = 4040            
s.connect((host_name, port))
temp = s.recv(1024).decode()
s.close()  

#start4 = time.time()
l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")

#end4 = time.time()
end = time.time()
#totaltime = (end3-start3) + (end4-start4)
totaltime = end-start
print("Authentication time: ", totaltime)

# with new server and client exp

In [ ]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import AES
import pyAesCrypt
import string
import secrets




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   

host_name = "DESKTOP-8RJUAE7"  
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 4040            
s.connect((host_name, port))
temp = s.recv(4096).decode()
s.close()  



#Registration
Di = 12345
PWi = "12345"

start = time.time()
start1 = time.time()

Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating  PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi

end1 = time.time()

start2 = time.time()
host_name = "DESKTOP-8RJUAE7"
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 4040
s.bind((host_name, port))
s.listen(5)
while True:
    c, addr = s.accept()
    c.send(temp.encode())
    c.close()
    break

end2 = time.time()

start3 = time.time()


host_name = "DESKTOP-8RJUAE7"
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 4040            
s.connect((host_name, port))
t_ = s.recv(4096).decode()
s.close()  
        
end3 = time.time()        
        
start4 = time.time()

l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

end4 = time.time()
end = time.time()


#Authentication
Di_auth = 1234
PWi_auth = "1234@1234"

startf = time.time()
start5 = time.time()

r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0


file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         

file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    check=int(ka)^int(T_sa)

    b = str(Vi_prime) + str(N_auth)
    # Encryption starts
    block_size = 16
    pad = lambda s: s + (block_size - len(s) % block_size) * chr(block_size - len(s) % block_size)
    iv = Random.new().read(AES.block_size) # Random IV

    def get_private_key(secret_key, salt):
        return hashlib.pbkdf2_hmac('SHA256', secret_key.encode(), salt.encode(), 65536, 32)

    def encrypt_with_AES(message, secret_key, salt):
        private_key = get_private_key(secret_key, salt)
        message = pad(message)
        cipher = AES.new(private_key, AES.MODE_CBC, iv)
        cipher_bytes = base64.b64encode(iv + cipher.encrypt(message))
        return bytes.decode(cipher_bytes)
    
    secret_key = str(Krs)
    salt = "salt"
    plain_text = str(b)
    
    cipher = encrypt_with_AES(plain_text, secret_key, salt)
    
    session_device = str(N_auth) + str(Krs)
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    
    end5 = time.time()
    
    start6 = time.time()
    
    
    #port = int(input("Enter the port number:"))
    host_name = "DESKTOP-8RJUAE7"
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    port = 4040
    s.bind((host_name, port))
    s.listen(5)
    while True:
        c, addr = s.accept()
        c.send(temp.encode())
        c.close()
        break
    
    end6 = time.time()

#receving h_n from gateway

start7 = time.time()

host_name = "DESKTOP-8RJUAE7"
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 4040            
s.connect((host_name, port))
temp = s.recv(4096).decode()
s.close()  

end7 = time.time()

start8 = time.time()

l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")

end8 = time.time()
endf = time.time()

rtotaltime = (end1 + end2 + end3 + end4) - (start1 + start2 + start3 + start4)  
atotaltime = (end5 + end6 + end7 + end8) - (start5 + start6 + start7 + start8)

#rtotaltime = (end1 + end4) - (start1 + start4)  
#atotaltime = (end5 + end8) - (start5 + start8)

rftotaltime = end - start  
aftotaltime = endf - startf


print("Registration time: ",rtotaltime)
print("Authentication time: ",atotaltime)
print("----------------------------")
print("Registration f time: ",rftotaltime)
print("Authentication f time: ",aftotaltime)

In [6]:
pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


# Device With Compression

In [7]:
import sys
import math
import sympy
import random
import time
import socket
import hashlib
import base64
from Crypto import Random
from Crypto.Cipher import AES
import pyAesCrypt
import string
import secrets
import os




def chebyshev(x,z,K):
    T0=1%K
    T1=x%K
    for i in range(2,z+1):
        T=(2*x*T1-T0)%K
        T0=T1
        T1=T
    return T
   

port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

start1 = time.time()
#start = time.time()    
#Registration
#Di=int(input("Di input:")) # Di input
Di = 12345
#PWi=input("PWi input:") # PWi input
PWi = "12345"

Ri=random.randint(0,sys.maxsize) # Large random number Ri
PIDi=Di^Ri # Generating PIDi
PIDi=str(PIDi)
t=time.time() # Gets current time stamp
t=str(t)
PINi=''.join([hex(ord(PWi[i%len(PWi)]) ^ ord(t[i%(len(t))]))[2:] for i in range(max(len(PWi), len(t)))]) #calculating PINi
PINi=int(PINi,base=16)
PINi=str(PINi) # Generating PWi
temp=PINi+","+PIDi

end1 = time.time()

#start = time.time()
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host_name, port))
    s.listen(5)
    while True:
        conn,addr=s.accept()
        with conn:
            #print("Connected to",addr)
            while True:
                data=conn.recv(1024)
                if not data:
                    break
                # print("Received from",data.decode())
                sent=temp.encode()
                try:
                    conn.sendall(sent)
                except ConnectionResetError as e:
                    break
        break

#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        t_=data.decode()
        break
        
        
start2 = time.time()
        
l=list(map(str,t_.split(",")))
T_s_from_gateway=l[2]
x_from_gateway=l[1]
Ck=l[0]
Ck=int(Ck)
temporary=PINi+PIDi
result = hashlib.sha256(temporary.encode())
ans=result.hexdigest()
answer=int(ans,base=16) # Output of Hash(PINi||PIDi)
PIDi=int(PIDi)
k=Ck^answer^PIDi # Calculating k
V=k^int(T_s_from_gateway) # Calculating V
Di=str(Di)
hash_input=Di+PWi
res= hashlib.sha256(hash_input.encode())
ans_1=res.hexdigest()
answer_1=int(ans_1,base=16) # Output of Hash(Di||PWi)
Vi=V^answer_1
file_storing=str(PWi)+"@"+str(Di)

list_=[str(Di)+"\t"+str(file_storing)+"\t"+str(k)+"\t"+str(x_from_gateway)+"\t"+str(T_s_from_gateway)+"\t"+str(Vi)+"\n"]
file=open("Store_file.txt","a") #storing values in the file
file.writelines(list_)
file.close()

#File Encryption
#alphabet = string.ascii_letters + string.digits
#password = ''.join(secrets.choice(alphabet) for i in range(256))
#pyAesCrypt.encryptFile("Store_file.txt", "Encrypted.txt.aes", password)


#end = time.time()
#totaltime = end-start
#print("Registration time: ", totaltime)

end2 = time.time()
totaltime = (end1-start1) + (end2-start2)
print("Registration time: ", totaltime)


start3 = time.time()
#start=time.time()
#Authentication
#Di_auth=int(input("Di input Authentication:"))
Di_auth = 1234
#PWi_auth=input("Enter the PWi@Di_auth:")
PWi_auth = "1234@1234"
r_auth=sympy.randprime(2,1000) #large random number R
N_auth=random.randint(0,sys.maxsize) # nonce
s=str(PWi_auth)
flag1 = 0
#File Decryption
#pyAesCrypt.decryptFile("Encrypted.txt.aes", "Decrypted.txt", password)

file1=open("Store_file.txt","r")
flag = 0
index = 0  

for lines in file1:  
    index += 1
    if s in lines:
        flag = 1
        break
         
#if flag == 0:
#    print('String', s , 'Not Found')
#else:
#    print('String', s, 'Found In Line', index)
file1.close()
if(flag != 0):
    file1=open("Store_file.txt","r")
    lines = file1.readlines()
    da,pa,ka,xa,T_sa,Via = map(str, lines[index-1].split("\t"))
    flag1 = 1
    #print(da,pa,ka,xa,T_sa,Via)
    file1.close()
else:
    print("No Details Found")

if(flag1 == 1):
    xa,ka,T_sa=int(xa),int(ka),int(T_sa)
    T_r=chebyshev(xa,r_auth,ka) # Calculating T_r value
    Krs=chebyshev(T_sa,r_auth,ka) # Calculating Krs

    string_partition=PWi_auth.partition('@')
    original_PWi=string_partition[0] # Gives the original PWi which is written in format PWi@Di

    hash_input_auth=str(Di_auth)+original_PWi
    res= hashlib.sha256(hash_input_auth.encode())
    ans_auth=res.hexdigest()
    answer_auth=int(ans_auth,base=16) # Output of Hash(Di||Pwi)
    Vi_prime=int(Via)^answer_auth
    #print("Vi_prime:",Vi_prime) # Calculate the value of Vi_prime
    check=int(ka)^int(T_sa)
    #print("check:",check) # Used to check whether the calculated Vi_prime is same is ka^T_sa

    b = str(Vi_prime) + str(N_auth)
    #print("b value: ", b)
    # Encryption starts
    block_size = 16
    pad = lambda s: s + (block_size - len(s) % block_size) * chr(block_size - len(s) % block_size)
    iv = Random.new().read(AES.block_size) # Random IV

    def get_private_key(secret_key, salt):
        return hashlib.pbkdf2_hmac('SHA256', secret_key.encode(), salt.encode(), 65536, 32)

    def encrypt_with_AES(message, secret_key, salt):
        private_key = get_private_key(secret_key, salt)
        message = pad(message)
        cipher = AES.new(private_key, AES.MODE_CBC, iv)
        cipher_bytes = base64.b64encode(iv + cipher.encrypt(message))
        return bytes.decode(cipher_bytes)
    
    secret_key = str(Krs)
    salt = "salt"
    plain_text = str(b)
    
    cipher = encrypt_with_AES(plain_text, secret_key, salt)
    #print("Cipher: " + cipher)
    
    session_device = str(N_auth) + str(Krs)
    print("session device",session_device)
    
    end3 = time.time()
    
    temp = str(cipher)+ "," + str(T_r) + "," + str(ka)+ "," +  str(T_sa) + "," + str(r_auth)
    #port = int(input("Enter the port number:"))
    port = 4040
    host_name = socket.gethostname()
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host_name, port))
        s.listen(5)
        while True:
            conn,addr=s.accept()
            with conn:
                #print("Connected to",addr)
                while True:
                    data=conn.recv(1024)
                    if not data:
                        break
                    #print("Received from",data.decode())
                    sent=temp.encode()
                    try:
                        conn.sendall(sent)
                    except ConnectionResetError as e:
                        break
            break

#receving h_n from gateway
#port = int(input("Enter the port number:"))
port = 4040
host_name = socket.gethostname()
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host_name, port))
    while True:
        #sent=input().encode()
        sent = "hi".encode()
        s.sendall(sent)
        data=s.recv(1024)
        '''print("Received",data.decode())'''
        temp=data.decode()
        break

start4 = time.time()
l=list(map(str,temp.split(",")))

h_n = l[0]

hash_input_auth = str(N_auth)
res= hashlib.sha256(hash_input_auth.encode())
ans_auth=res.hexdigest()
h_n_prime = int(ans_auth,base=16) # Output of Hash(N_auth)

if(h_n == str(h_n_prime)):
    print("Yes, Gateway is verified by the device")

end4 = time.time()
#end = time.time()
totaltime = (end3-start3) + (end4-start4)
#totaltime = end-start
print("Authentication time: ", totaltime)


#ZLIB Code
import zlib, base64
file = open('dataset.txt','r')
text = file.read()
file.close()
#print("Uncompressed Text: \n"+text)
# encoding the text
code =  base64.b64encode(zlib.compress(text.encode('utf-8'),9))
code = code.decode('utf-8')
f=open('compressed.txt','w')
f.write(code)
f.close()

#File Encryption
alphabet = string.ascii_letters + string.digits
password = session_device
pyAesCrypt.encryptFile("compressed.txt", "Enc_compressed.txt.aes", password)

os.remove("compressed.txt")



import socket
import os

class Client:
    def __init__(self):
        self.s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.connect_to_server()

    def connect_to_server(self):
        self.target_ip = socket.gethostname()
        self.target_port = 4444

        self.s.connect((self.target_ip,int(self.target_port)))

        self.main()

    def main(self):
        while 1:
            file_name = input('Enc_compressed.txt.aes')
            self.s.send(file_name.encode())

            confirmation = self.s.recv(1024)
            if confirmation.decode() == "file-doesn't-exist":
                print("File doesn't exist on server.")

                self.s.shutdown(socket.SHUT_RDWR)
                self.s.close()

            else:        
                write_name = 'from_server '+file_name
                if os.path.exists(write_name): os.remove(write_name)

                with open(write_name,'wb') as file:
                    while 1:
                        data = self.s.recv(1024)

                        if not data:
                            break

                        file.write(data)

                print(file_name,'successfully downloaded.')
                break

                self.s.shutdown(socket.SHUT_RDWR)
                self.s.close()
                
client = Client()


ModuleNotFoundError: No module named 'Crypto'

# File Encryption and Decryption

In [ ]:
import pyAesCrypt
import string
import secrets
import os

#File Encryption
alphabet = string.ascii_letters + string.digits
password = "krishna"
pyAesCrypt.encryptFile("Store_file1.txt", "Store_file1.txt.aes", password)

os.remove("Store_file1.txt")

#File Decryption
#pyAesCrypt.decryptFile("Store_file1.txt.aes", "Store_file1.txt", password)

In [ ]:
#File Decryption
pyAesCrypt.decryptFile("Store_file1.txt.aes", "Store_file1.txt", password)

In [ ]:
import socket
import threading
import os

class Server:
    def __init__(self):
        self.s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.accept_connections()
    
    def accept_connections(self):
        ip = socket.gethostbyname(socket.gethostname())
        port = 4444

        self.s.bind((ip,port))
        self.s.listen(100)

        print('Running on IP: '+ip)
        print('Running on port: '+str(port))

        while 1:
            c, addr = self.s.accept()
            print(c)
            
            threading.Thread(target=self.handle_client,args=(c,addr,)).start()

    def handle_client(self,c,addr):
        data = c.recv(1024).decode()
    
        if not os.path.exists(data):
            c.send("file-doesn't-exist".encode())

        else:
            c.send("file-exists".encode())
            print('Sending',data)
            if data != '':
                file = open(data,'rb')
                data = file.read(1024)
                while data:
                    c.send(data)
                    data = file.read(1024)

                c.shutdown(socket.SHUT_RDWR)
                c.close()
                

server = Server()